In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split


In [25]:

batch_size_train = 64
batch_size_test = 64



In [26]:
#其中values表示的是去掉表头和索引，只取数据
minst_data = pd.read_csv("./Dataset.csv", header=None).values
data = minst_data[:, :16]
label = minst_data[:, -1]


In [27]:
data[0, :]

array([0.478125  , 0.4125    , 0.434375  , 0.45625   , 0.45625   ,
       0.390625  , 0.390625  , 0.        , 0.71666667, 0.80416667,
       0.84583333, 0.88958333, 0.86875   , 0.78125   , 0.86875   ,
       0.        ])

In [28]:
label[0]

2.0

In [29]:
#使用线性模型
data_train, data_test, label_train, label_test = train_test_split(data, label, test_size=0.2, random_state=2020)
print(data_train.shape, data_test.shape)
print(label_train.shape, label_test.shape)


(869, 16) (218, 16)
(869,) (218,)


In [30]:
import torch
from torch.utils.data import TensorDataset,DataLoader
random_seed = 1
torch.cuda.manual_seed(random_seed)

data_train = torch.from_numpy(data_train).float()
label_train = torch.from_numpy(label_train).long()
#TensorDataset相当于zip功能，将data和label按照第一个维度打包成两个tensor，DataLoader主要用来进行batch_size的划分，drop表示是否去除最后不满足一个batch_size大小的数据
data_train = DataLoader(TensorDataset(data_train, label_train), batch_size=batch_size_train, shuffle=True, drop_last=True)

data_test = torch.from_numpy(data_test).float()
label_test = torch.from_numpy(label_test).long()
data_test = DataLoader(TensorDataset(data_test, label_test), batch_size=batch_size_test, shuffle=True, drop_last=True)


In [31]:
print(type(torch.tensor(data)))
print(torch.tensor(data).shape)
print(type(data_train))

<class 'torch.Tensor'>
torch.Size([1087, 16])
<class 'torch.utils.data.dataloader.DataLoader'>


In [38]:
import torch
import time
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
from Linear_for_openpose import LinearNet
batch_size = 64
##设置本次要训练用的模型
train_name = 'Linear_for_openpose_by_BN'
print("train_name:" + train_name)
##设置模型保存名称
savemodel_name = train_name + ".pt"
print("savemodel_name:" + savemodel_name)
##设置初始预测率,用于判断高于当前预测率的保存模型
toppredicted = 0.0
##设置学习率
learnrate = 0.05 
##设置动量值，如果上一次的momentnum与本次梯度方向是相同的，梯度下降幅度会拉大，起到加速迭代的作用
momentnum = 0.5

train_name:Linear_for_openpose_by_BN
savemodel_name:Linear_for_openpose_by_BN.pt


In [39]:
##设置选择训练模型,因为python用的是3.9,用不了match case语法
def switch(train_name):
    if train_name == 'Linear_for_openpose_by_BN':
        return LinearNet()
 
 
 
 
##定义训练模型
class Net(torch.nn.Module):
    def __init__(self, train_name):
        super(Net, self).__init__()
        self.model = switch(train_name= train_name)
        self.criterion = self.model.criterion
 
 
    def forward(self, x):
        x = self.model(x)
        return x

In [40]:
model = Net(train_name)
##加入判断是CPU训练还是GPU训练
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
 
 
##优化器 
optimizer = optim.SGD(model.parameters(), lr= learnrate, momentum= momentnum)

In [41]:

train_dataloader = data_train
test_dataloader = data_test


In [42]:
 
##训练函数
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_dataloader, 0):
        inputs, target = data
        ##加入CPU和GPU选择
        inputs, target = inputs.to(device), target.to(device)
 
 
        optimizer.zero_grad()
 
 
        #前馈，反向传播，更新
        outputs = model(inputs)
        loss = model.criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
 
        running_loss += loss.item()
        ##计算每300次打印一次学习效果
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0
 
 
 
 
def test():
    correct = 0 
    total = 0
    ##with这里标记是不再计算梯度
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            ##加入CPU和GPU选择
            inputs, labels = inputs.to(device), labels.to(device)
 
 
 
 
            outputs = model(inputs)
            ##预测返回的是两列，第一列是下标就是0-9的值，第二列为预测值，下面的dim=1就是找维度1（第二列）最大值输出
            _, predicted = torch.max(outputs.data, dim=1)
 
 
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    currentpredicted = (100 * correct / total)
    ##用global声明toppredicted,用于在函数内部修改在函数外部声明的全局变量，否则报错
    global toppredicted
    ##当预测率大于原来的保存模型
    if currentpredicted > toppredicted:
        toppredicted = currentpredicted
        torch.save(model.state_dict(), savemodel_name)
        print(savemodel_name+" saved, currentpredicted:%d %%" % currentpredicted)
 
 
    print('Accuracy on test set: %d %%' % currentpredicted)        
 

In [43]:
timestart = time.time()
for epoch in range(200):
    train(epoch)
    test()
timeend = time.time() - timestart
print("use time: {:.0f}m {:.0f}s".format(timeend // 60, timeend % 60))

Linear_for_openpose_by_BN.pt saved, currentpredicted:52 %
Accuracy on test set: 52 %
Linear_for_openpose_by_BN.pt saved, currentpredicted:53 %
Accuracy on test set: 53 %
Accuracy on test set: 51 %
Accuracy on test set: 50 %
Accuracy on test set: 52 %
Accuracy on test set: 51 %
Accuracy on test set: 51 %
Accuracy on test set: 53 %
Accuracy on test set: 51 %
Accuracy on test set: 50 %
Accuracy on test set: 51 %
Accuracy on test set: 53 %
Accuracy on test set: 51 %
Accuracy on test set: 53 %
Accuracy on test set: 50 %
Accuracy on test set: 52 %
Linear_for_openpose_by_BN.pt saved, currentpredicted:53 %
Accuracy on test set: 53 %
Accuracy on test set: 51 %
Accuracy on test set: 52 %
Accuracy on test set: 52 %
Accuracy on test set: 52 %
Accuracy on test set: 51 %
Linear_for_openpose_by_BN.pt saved, currentpredicted:54 %
Accuracy on test set: 54 %
Accuracy on test set: 52 %
Accuracy on test set: 51 %
Accuracy on test set: 50 %
Accuracy on test set: 52 %
Accuracy on test set: 52 %
Accuracy on 

## 已完成训练

如何载入模型？

In [ ]:
#保存时
PATH = './model.pth'

torch.save(model,PATH)




In [ ]:
#加载时
model = torch.load('model.pth')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
##优化器 
optimizer = optim.SGD(model.parameters(), lr= learnrate, momentum= momentnum)

In [ ]:

correct = 0 
total = 0
##with这里标记是不再计算梯度
with torch.no_grad():
    inputs, labels = torch.tensor(linear_data), torch.tensor(label)
    inputs = inputs.to(torch.float32)
    ##加入CPU和GPU选择
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    ##预测返回的是两列，第一列是下标就是0-9的值，第二列为预测值，下面的dim=1就是找维度1（第二列）最大值输出
    _, predicted = torch.max(outputs.data, dim=1)
    print(inputs.shape)
    print(labels)
    print(predicted)
    # 前columns设置标签名（预测结果是两列数，就加两个标签），后data为待保存的数据
    test=pd.DataFrame(columns=['start'], data=predicted.cpu())
    #删除第一行
    # test = test.drop(0)
    # 2.数据保存，index表示是否显示行名，sep数据分开符
    test.to_csv('save1.csv', index=False, sep=',')
  
